# Shallow Water Equations

The shallow water equations in one space dimension are
$$
\partial_t \underbrace{\begin{pmatrix} h \\ hv \end{pmatrix}}_{=u}
+ \partial_x \underbrace{\begin{pmatrix} hv \\ h v^2 + \frac{1}{2} g h^2 \end{pmatrix}}_{=f(u)}
= 0,
$$
where $h$ is the water height, $v$ the velocity, and $g$ the gravitational constant.

In [ ]:
using LaTeXStrings
using Plots; pyplot()

using HyperbolicDiffEq

## Dam Break

Example 5.20 of Holden & Risebro, "Front Tracking for Hyperbolic Conservation Laws", 2002.

In [ ]:
model = ShallowWater(1.)
uₗ = ShallowWaterVar1D(1., 0.)
uᵣ = ShallowWaterVar1D(0., 0.)
prob = RiemannProblem(model, uₗ, uᵣ)
sol = solve(prob)
plot(sol)

## Moses' First Problem

Example 5.21 of Holden & Risebro, "Front Tracking for Hyperbolic Conservation Laws", 2002.

In [ ]:
model = ShallowWater(1.)
uₗ = ShallowWaterVar1D(1., -2.5)
uᵣ = ShallowWaterVar1D(1.,  2.5)
prob = RiemannProblem(model, uₗ, uᵣ)
sol = solve(prob)
plot(sol)

## Moses' Second Problem

Example 5.22 of Holden & Risebro, "Front Tracking for Hyperbolic Conservation Laws", 2002.

In [ ]:
model = ShallowWater(1.)
uₗ = ShallowWaterVar1D(1., 0.)
uₘ = ShallowWaterVar1D(0., 0.)
uᵣ = ShallowWaterVar1D(1., 0.)
prob1 = RiemannProblem(model, uₗ, uₘ)
prob2 = RiemannProblem(model, uₘ, uᵣ, 1)
prob = prob1 * prob2
sol = solve(prob)
plot(sol)

# Two Shocks

In [ ]:
model = ShallowWater(1.)
uₗ = ShallowWaterVar1D(1., 10.)
uᵣ = ShallowWaterVar1D(1., 0.)
prob = RiemannProblem(model, uₗ, uᵣ)
sol = solve(prob)
plot(sol)

# First Order Finite Volume Methods

In [ ]:
using Revise
using HyperbolicDiffEq
using OrdinaryDiffEq, DiffEqCallbacks
using LaTeXStrings, Plots; pyplot()

xmin, xmax = -2., 2.
N = 10^3
usethreads = true

balance_law = ShallowWater()
numflux = local_lax_friedrichs # godunov, local_lax_friedrichs

# set up a periodic problem
u1 = ShallowWaterVar1D(1., 10.)
u2 = ShallowWaterVar1D(1., 0.)
u_ana = solve(RiemannProblem(balance_law, u1, u2, -0.5) * RiemannProblem(balance_law, u2, u1, 0.5))
tspan = (0., 1.)
u₀func(x) = u_ana(tspan[1], x)

# create an ODE
mesh = UniformPeriodicMesh1D(xmin, xmax, N)
fv = FirstOrderFV(balance_law, mesh, numflux, usethreads)
ode = semidiscretise(fv, u₀func, tspan)

# solve the ODE sing either an adaptive time step (CFL) or a fixed one
#cb = StepsizeLimiter((t,u)->max_dt(t,u,fv), safety_factor=1, max_step=true)
#sol = solve(ode, Euler(), dt=1, callback=cb, save_everystep=false)
sol = solve(ode, Euler(), dt=max_dt(tspan[1], ode.u0, fv), save_everystep=false)

#plot(xguide=L"x", yguide=L"u")
#plot!(evaluate_coefficients(sol[1], mesh), label=L"u_0")
#plot!(evaluate_coefficients(sol[end], mesh), label=L"u_\mathrm{num}")
#x = linspace(xmin, xmax, 10^3)
#plot!(x, u_ana.(tspan[end], x), label=L"u_\mathrm{ana}", legend=true)